In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/Desktop/Model_preds/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")
dat1=pd.DataFrame(dat)
dat1=dat1.iloc[152:,:].reset_index(drop=True)

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r=Y_r.iloc[:,:]

Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

/home/ciaran/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Single trade each day version, 1.25MW battery (nominal charge 20% or 0.25 MW)

In [32]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/Desktop/Model_preds/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")
dat1=pd.DataFrame(dat)
dat1=dat1.iloc[152:,:].reset_index(drop=True)

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r=Y_r.iloc[:,:]

Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds, eff_1, eff_2):
    prices = []
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index)] 
        min_price_index = current_Q_B['Price'].idxmin()
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > min_price_index)] 
        
        min_price_index1 = None
        max_price_index1 = None

        if len(remaining_prices_0_B) > 0:
            min_price_index1 = remaining_prices_0_B['Price'].idxmin()  
        if len(remaining_prices_0_A) > 0:
            max_price_index1 = remaining_prices_0_A['Price'].idxmax() 

        if max_price_index is not None and min_price_index1 is not None and max_price_index1 is not None:
            if (current_Q_A.loc[max_price_index, 'Price'] - current_Q_B.loc[min_price_index1, 'Price']) > (current_Q_A.loc[max_price_index1, 'Price'] - current_Q_B.loc[min_price_index, 'Price']):
                profit = ((current_df.loc[max_price_index, 'Price']) * eff_1) - ((current_df.loc[min_price_index1, 'Price']) / eff_2)
                prices.append((min_price_index1, current_df.loc[min_price_index1, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit))
            else:
                profit = ((current_df.loc[max_price_index1, 'Price']) * eff_1) - ((current_df.loc[min_price_index, 'Price']) / eff_2)
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index1, current_df.loc[max_price_index1, 'Price'], profit))
        elif max_price_index is not None and min_price_index1 is not None:
            profit = ((current_df.loc[max_price_index, 'Price']) * eff_1) - ((current_df.loc[min_price_index1, 'Price']) / eff_2)
            prices.append((min_price_index1, current_df.loc[min_price_index1, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit))
        elif max_price_index1 is not None and min_price_index is not None:
            profit = ((current_df.loc[max_price_index1, 'Price']) * eff_1) - ((current_df.loc[min_price_index, 'Price']) / eff_2)
            prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index1, current_df.loc[max_price_index1, 'Price'], profit))
            
    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice', 'maxPriceIndex', 'maxPrice', 'profit'])



r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_dam_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98)
r_dam_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_dam_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_dam_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)

PF_DAM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r,        eff_1=0.8, eff_2=0.98)

print(np.round(sum(r_dam_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_DAM.iloc[:,4:5].values),2))

[16733.87]
[16635.88]
[16978.17]
[16716.54]
[15535.64]
[16966.1]
[16642.67]
[19078.85]


In [33]:
r_dam_30_50

,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit
0,5,12.26,18,28.66,10.417796
1,28,12.89,42,36.00,15.646939
2,50,16.23,65,37.16,13.166776
3,85,15.00,90,37.00,14.293878
4,99,17.40,104,35.19,10.396898
...,...,...,...,...,...
358,8640,88.64,8659,180.29,53.783020
359,8667,86.24,8672,246.11,108.888000
360,8692,87.65,8696,252.37,112.457224
361,8715,77.62,8731,117.03,14.419918


In [15]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/Desktop/Model_preds/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")
dat1=pd.DataFrame(dat)
dat1=dat1.iloc[152:,:].reset_index(drop=True)

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r=Y_r.iloc[:,:]

Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2):
    prices = []
    level_0_values = df['level_0'].unique()
    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]        
        max_price_index = current_Q_A['Price'].idxmax()       
        
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index)]       

        if len(remaining_prices_0_B) > 0:
            min_price_index = remaining_prices_0_B['Price'].idxmin()            
            profit = ((current_df.loc[max_price_index, 'Price'])*eff_1) - ((current_df.loc[min_price_index, 'Price'])/eff_2)
            prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit))
        else:
            continue                                       

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit'])



r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_dam_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98)
r_dam_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_dam_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_dam_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)

PF_DAM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r,        eff_1=0.8, eff_2=0.98)

print(np.round(sum(r_dam_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_DAM.iloc[:,4:5].values),2))


[16730.31]
[16629.76]
[16984.07]
[16694.22]
[15530.66]
[16947.98]
[16608.03]
[19064.92]


In [16]:
r_dam_50_50

,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit
0,5,12.26,17,31.66,12.817796
1,28,12.89,42,36.00,15.646939
2,50,16.23,65,37.16,13.166776
3,85,15.00,90,37.00,14.293878
4,99,17.40,104,35.19,10.396898
...,...,...,...,...,...
356,8640,88.64,8659,180.29,53.783020
357,8667,86.24,8673,219.04,87.232000
358,8692,87.65,8696,252.37,112.457224
359,8715,77.62,8721,128.52,23.611918


In [25]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/Desktop/Model_preds/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")
dat1=pd.DataFrame(dat)
dat1=dat1.iloc[152:,:].reset_index(drop=True)

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r=Y_r.iloc[:,:]

Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds, eff_1, eff_2):
    prices = []
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index)] 
        min_price_index = current_Q_B['Price'].idxmin()
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > min_price_index)] 
        
        min_price_index1 = None
        max_price_index1 = None

        if len(remaining_prices_0_B) > 0:
            min_price_index1 = remaining_prices_0_B['Price'].idxmin()  
        if len(remaining_prices_0_A) > 0:
            max_price_index1 = remaining_prices_0_A['Price'].idxmax() 

        if max_price_index is not None and min_price_index1 is not None and max_price_index1 is not None:
            if (current_Q_A.loc[max_price_index, 'Price'] - current_Q_B.loc[min_price_index1, 'Price']) > (current_Q_A.loc[max_price_index1, 'Price'] - current_Q_B.loc[min_price_index, 'Price']):
                profit = ((current_df.loc[max_price_index, 'Price'])*eff_1) - ((current_df.loc[min_price_index1, 'Price'])/eff_2)
                prices.append((min_price_index1, current_df.loc[min_price_index1, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit))
            else:
                profit = ((current_df.loc[max_price_index1, 'Price'])*eff_1) - ((current_df.loc[min_price_index, 'Price'])/eff_2)
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index1, current_df.loc[max_price_index1, 'Price'], profit))
            
    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice', 'maxPriceIndex', 'maxPrice', 'profit'])



r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_dam_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98)
r_dam_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_dam_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_dam_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)

PF_DAM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r,        eff_1=0.8, eff_2=0.98)

print(np.round(sum(r_dam_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_DAM.iloc[:,4:5].values),2))

[15148.54]
[14350.73]
[15403.12]
[15684.3]
[14672.17]
[15951.31]
[15729.14]
[16482.04]


In [ ]:
16730.31	16629.76	16984.07	16694.22	15530.66	16947.98	16608.03	19064.92	16589.29


In [21]:
r_dam_50_50

,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit
0,5,12.26,17,31.66,12.817796
1,28,12.89,42,36.00,15.646939
2,50,16.23,65,37.16,13.166776
3,85,15.00,90,37.00,14.293878
4,158,-27.93,163,12.78,38.724000
...,...,...,...,...,...
301,8618,65.52,8635,129.81,36.990857
302,8640,88.64,8659,180.29,53.783020
303,8667,86.24,8673,219.04,87.232000
304,8715,77.62,8721,128.52,23.611918


In [30]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/Desktop/Model_preds/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")
dat1=pd.DataFrame(dat)
dat1=dat1.iloc[152:,:].reset_index(drop=True)

column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 24)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 24)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 24)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 24)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 24)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 24)]
Y_r=dat1[column_names].dropna().stack().reset_index()
Y_r=Y_r.iloc[:,:]

Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds, eff_1, eff_2):
    prices = []
    level_0_values = df['level_0'].unique()

    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]
        
        max_price_index = current_Q_A['Price'].idxmax()
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index)] 
        min_price_index = current_Q_B['Price'].idxmin()
        remaining_prices_0_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > min_price_index)] 
        
        min_price_index1 = None
        max_price_index1 = None

        if len(remaining_prices_0_B) > 0:
            min_price_index1 = remaining_prices_0_B['Price'].idxmin()  
        if len(remaining_prices_0_A) > 0:
            max_price_index1 = remaining_prices_0_A['Price'].idxmax() 

        if max_price_index is not None and min_price_index1 is not None and max_price_index1 is not None:
            if (current_Q_A.loc[max_price_index, 'Price'] - current_Q_B.loc[min_price_index1, 'Price']) > (current_Q_A.loc[max_price_index1, 'Price'] - current_Q_B.loc[min_price_index, 'Price']):
                profit = ((current_df.loc[max_price_index, 'Price']) * eff_1) - ((current_df.loc[min_price_index1, 'Price']) / eff_2)
                prices.append((min_price_index1, current_df.loc[min_price_index1, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit))
            else:
                profit = ((current_df.loc[max_price_index1, 'Price']) * eff_1) - ((current_df.loc[min_price_index, 'Price']) / eff_2)
                prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index1, current_df.loc[max_price_index1, 'Price'], profit))
        elif max_price_index is not None and min_price_index1 is not None:
            profit = ((current_df.loc[max_price_index, 'Price']) * eff_1) - ((current_df.loc[min_price_index1, 'Price']) / eff_2)
            prices.append((min_price_index1, current_df.loc[min_price_index1, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit))
        elif max_price_index1 is not None and min_price_index is not None:
            profit = ((current_df.loc[max_price_index1, 'Price']) * eff_1) - ((current_df.loc[min_price_index, 'Price']) / eff_2)
            prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index1, current_df.loc[max_price_index1, 'Price'], profit))
            
    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice', 'maxPriceIndex', 'maxPrice', 'profit'])



r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_dam_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98)
r_dam_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_dam_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_dam_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)

PF_DAM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r,        eff_1=0.8, eff_2=0.98)

print(np.round(sum(r_dam_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_DAM.iloc[:,4:5].values),2))

[16733.87]
[16635.88]
[16978.17]
[16716.54]
[15535.64]
[16966.1]
[16642.67]
[19078.85]


In [31]:
r_dam_50_50

,minPriceIndex,minPrice,maxPriceIndex,maxPrice,profit
0,5,12.26,17,31.66,12.817796
1,28,12.89,42,36.00,15.646939
2,50,16.23,65,37.16,13.166776
3,85,15.00,90,37.00,14.293878
4,99,17.40,104,35.19,10.396898
...,...,...,...,...,...
358,8640,88.64,8659,180.29,53.783020
359,8667,86.24,8673,219.04,87.232000
360,8692,87.65,8696,252.37,112.457224
361,8715,77.62,8721,128.52,23.611918


In [ ]:
16730.31	16629.76	16984.07	16694.22	15530.66	16947.98	16608.03	19064.92	16589.29


In [ ]:
print(np.round(sum(r_dam_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_dam_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_DAM.iloc[:,4:5].values),2))



In [ ]:
date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
dat1 = pd.read_csv("/home/ciaran/rf_Q_1-12.csv")
dat1=pd.DataFrame(dat1)
column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r=dat1[column_names].dropna().stack().reset_index()
# YY_test= Y_r.iloc[:, 2:]
Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

In [9]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure



date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_1-12.csv")
# dat = pd.read_csv("/home/ciaran/rf_Q_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_1-12.csv")
dat = pd.read_csv("/home/ciaran/MH_Q_1-12.csv")
dat1=pd.DataFrame(dat)
column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r=dat1[column_names].dropna().stack().reset_index()
# YY_test= Y_r.iloc[:, 2:]
Y_r["Price"]=Y_r.iloc[:,2:3]
Q_10["Price"]=Q_10.iloc[:,2:3]
Q_30["Price"]=Q_30.iloc[:,2:3]
Q_50["Price"]=Q_50.iloc[:,2:3]
Q_70["Price"]=Q_70.iloc[:,2:3]
Q_90["Price"]=Q_90.iloc[:,2:3]

def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2):
    prices = []
    level_0_values = df['level_0'].unique()
    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]        
        max_price_index = current_Q_A['Price'].idxmax()       
        
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index)]
        

        if len(remaining_prices_0_B) > 0:
            min_price_index = remaining_prices_0_B['Price'].idxmin()            
            profit = ((current_df.loc[max_price_index, 'Price'])*eff_1) - ((current_df.loc[min_price_index, 'Price'])/eff_2)
            prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit))
        else:
            continue  

                         
                    
               

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit'])


r_bm_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_bm_10_30=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_30, eff_1=0.8, eff_2=0.98)
r_bm_30_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_bm_50_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_bm_70_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_70, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)
r_bm_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)
r_bm_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)

PF_BM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r,        eff_1=0.8, eff_2=0.98)

print(np.round(sum(r_bm_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_BM.iloc[:,4:5].values),2))

[-111.59]
[-6040.22]
[-4515.88]
[761.52]
[55.6]
[-4110.33]
[-4129.92]
[69715.95]


In [ ]:
print(np.round(sum(r_bm_50_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_30.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_50.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_50_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_70_90.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_30_70.iloc[:,4:5].values),2))
print(np.round(sum(r_bm_10_90.iloc[:,4:5].values),2))
print(np.round(sum(PF_BM.iloc[:,4:5].values),2))

In [ ]:
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2):
    prices = []
    level_0_values = df['level_0'].unique()
    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]        
        max_price_index = current_Q_A['Price'].idxmax()       
        
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index)]
        

        if len(remaining_prices_0_B) > 0:
            min_price_index = remaining_prices_0_B['Price'].idxmin()            
            profit = ((current_df.loc[max_price_index, 'Price'])*eff_1) - ((current_df.loc[min_price_index, 'Price'])/eff_2)
            prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit))
        else:
            continue  

                         
                    
               

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit'])


r_bm_PF=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r, eff_1=0.8, eff_2=0.98)
r_bm_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=0.8, eff_2=0.98)
r_bm_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=0.8, eff_2=0.98)
r_bm_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=0.8, eff_2=0.98)

In [ ]:
print(sum(r_bm_50_50.iloc[:,4:5].values))
r_bm_50_50

In [ ]:
def run_electricity_strategy(df, Q_A_Preds, Q_B_Preds,  eff_1, eff_2, capacity,charge_level, ramp_rate, min_charge_level):
    prices = []
    charge_level = charge_level    
    level_0_values = df['level_0'].unique()
    for level_0 in level_0_values:
        current_df = df[df['level_0'] == level_0]
        current_Q_A = Q_A_Preds[Q_A_Preds['level_0'] == level_0]
        current_Q_B = Q_B_Preds[Q_B_Preds['level_0'] == level_0]        
        max_price_index = current_Q_A['Price'].idxmax()       
        
        remaining_prices_0_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index)]
        
        remaining_prices_1_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > max_price_index)]

        if len(remaining_prices_0_B) > 0:
            min_price_index = remaining_prices_0_B['Price'].idxmin()            
            profit = ((current_df.loc[max_price_index, 'Price'])*eff_1) - ((current_df.loc[min_price_index, 'Price'])/eff_2)
            prices.append((min_price_index, current_df.loc[min_price_index, 'Price'], max_price_index, current_df.loc[max_price_index, 'Price'], profit))
        else:
            continue  

                        
            remaining_prices_0a_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index < min_price_index)]
            max_price_index_0a    = remaining_prices_0a_A['Price'].idxmax()
            remaining_prices_0a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index_0a)]

            if len(remaining_prices_0a_B) > 0:
                min_price_index_0a = remaining_prices_0a_B['Price'].idxmin()
                profit = ((current_df.loc[max_price_index_0a, 'Price'])*eff_1) - ((current_df.loc[min_price_index_0a, 'Price'])/eff_2)
                prices.append((min_price_index_0a, current_df.loc[min_price_index_0a, 'Price'], max_price_index_0a, current_df.loc[max_price_index_0a, 'Price'], profit))              
            else:
                continue
                
                   
                remaining_prices_0b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index < min_price_index_0a)]
                max_price_index_0b    = remaining_prices_0b_A['Price'].idxmax()
                remaining_prices_0b_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index < max_price_index_0b)]

                if len(remaining_prices_0b_B) > 0:
                    min_price_index_0b = remaining_prices_0b_B['Price'].idxmin()
                    profit = ((current_df.loc[max_price_index_0b, 'Price'])*eff_1) - ((current_df.loc[min_price_index_0b, 'Price'])/eff_2)
                    prices.append((min_price_index_0b, current_df.loc[min_price_index_0b, 'Price'], max_price_index_0b, current_df.loc[max_price_index_0b, 'Price'], profit))              
                else:
                    continue    
                    

                    
        if len(remaining_prices_1_A) > 0:
            max_price_index_1a    = remaining_prices_1_A['Price'].idxmax()
            remaining_prices_1a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > max_price_index) & (Q_B_Preds.index < max_price_index_1a)]
            if len(remaining_prices_1a_B) > 0:  
                min_price_index_1a = remaining_prices_1a_B['Price'].idxmin() 
                profit = ((current_df.loc[max_price_index_1a, 'Price'])*eff_1) - ((current_df.loc[min_price_index_1a, 'Price'])/eff_2)
                prices.append((min_price_index, current_df.loc[min_price_index_1a, 'Price'], max_price_index, current_df.loc[max_price_index_1a, 'Price'], profit))        
            else:
                continue 
        else:
            continue  

                        
            remaining_prices_1b_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > max_price_index) & (Q_A_Preds.index < min_price_index_1a)]
            max_price_index_1b    = remaining_prices_1b_A['Price'].idxmax()
            remaining_prices_1a_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > max_price_index) & (Q_B_Preds.index < max_price_index_1b)]

            if len(remaining_prices_1a_B) > 0:
                min_price_index_1b = remaining_prices_1a_B['Price'].idxmin()
                profit = ((current_df.loc[max_price_index_1b, 'Price'])*eff_1) - ((current_df.loc[min_price_index_1b, 'Price'])/eff_2)
                prices.append((min_price_index_1b, current_df.loc[min_price_index_1b, 'Price'], max_price_index_1b, current_df.loc[max_price_index_1b, 'Price'], profit))              
            else:
                continue
                
                   
                remaining_prices_2_A = Q_A_Preds[(Q_A_Preds['level_0'] == level_0) & (Q_A_Preds.index > max_price_index_1a)]
                max_price_index_2    = remaining_prices_2_A['Price'].idxmax()
                remaining_prices_2_B = Q_B_Preds[(Q_B_Preds['level_0'] == level_0) & (Q_B_Preds.index > max_price_index_1a) & (Q_B_Preds.index < max_price_index_2)]

                if len(remaining_prices_2_B) > 0:
                    min_price_index_2 = remaining_prices_2_B['Price'].idxmin()
                    profit = ((current_df.loc[max_price_index_2, 'Price'])*eff_1) - ((current_df.loc[min_price_index_2, 'Price'])/eff_2)
                    prices.append((min_price_index_2, current_df.loc[min_price_index_2, 'Price'], max_price_index_2, current_df.loc[max_price_index_2, 'Price'], profit))              
                else:
                    continue               
                    
               

    return pd.DataFrame(prices, columns=['minPriceIndex', 'minPrice','maxPriceIndex', 'maxPrice', 'profit'])



r_dam_50_50=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_50, Q_B_Preds=Q_50, eff_1=1, eff_2=1, capacity=5,charge_level=3, ramp_rate=1, min_charge_level=0)
r_dam_10_90=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_10, Q_B_Preds=Q_90, eff_1=1, eff_2=1, capacity=5,charge_level=3, ramp_rate=1, min_charge_level=0)
r_dam_30_70=run_electricity_strategy(df=Y_r, Q_A_Preds=Q_30, Q_B_Preds=Q_70, eff_1=1, eff_2=1, capacity=5,charge_level=3, ramp_rate=1, min_charge_level=0)
PF_DAM=run_electricity_strategy(df=Y_r, Q_A_Preds=Y_r, Q_B_Preds=Y_r, eff_1=1, eff_2=1, capacity=5,charge_level=3, ramp_rate=1, min_charge_level=0)

In [ ]:
print(sum(r_dam_50_50.iloc[:,4:5].values))
r_dam_50_50

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


date_format="%d/%m/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lgbm_Q_DAM_1-12.csv")
dat = pd.read_csv("/home/ciaran/rf_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_DAM_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_DAM_1-12.csv")

dat1=pd.DataFrame(dat)


In [ ]:
column_names = ['EURPrices+{}_Forecast_10'.format(i) for i in range(0, 23)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_30'.format(i) for i in range(0, 23)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_50'.format(i) for i in range(0, 23)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_70'.format(i) for i in range(0, 23)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}_Forecast_90'.format(i) for i in range(0, 23)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['EURPrices+{}'.format(i) for i in range(0, 23)]
Y_r=dat1[column_names].dropna().stack().reset_index()

# plt.plot(Y_r.iloc[:, 2:])
# Y_r.iloc[:, 2:].shape
# Q_30

In [ ]:
def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 100000
    profits = 0
    trade_log = []
    max_units = 1

    num_trades = 0
    latest_buy_timestamp = 0

    for i in range(0, len(quantile_70_predictions), 24):
        max_profit = 0
        best_buy_timestamp = i
        best_sell_timestamp = i

        for j in range(i, min(i+24, len(quantile_70_predictions))):
            if j <= latest_buy_timestamp:
                continue

            for k in range(j, min(i+24, len(quantile_70_predictions))):
                if k - j > 48:
                    break
                if quantile_70_predictions[j] < quantile_30_predictions[k]:
                    purchase_price = real_prices[j]
                    sell_price = real_prices[k]
                    units = min(max_units, abs(funds // purchase_price))
                    if purchase_price < 0:
                        units = max_units
                    profit = (units * (sell_price - purchase_price)) * 0.8

                    if profit > max_profit:
                        max_profit = profit
                        best_buy_timestamp = j
                        best_sell_timestamp = k

        if max_profit > 0:
            purchase_price = real_prices[best_buy_timestamp]
            sell_price = real_prices[best_sell_timestamp]
            units = min(max_units, abs(funds // purchase_price))
            if purchase_price < 0:
                units = max_units
            profit = (units * (sell_price - purchase_price)) * 0.8

            profits += profit
            funds = funds 

            trade_log.append((best_buy_timestamp, best_sell_timestamp, purchase_price, sell_price, units, profit))

            num_trades += 1
            latest_buy_timestamp = best_buy_timestamp

    return profits, trade_log, num_trades, units, funds



quantile_30_predictions = np.array(pd.DataFrame(Q_30.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Q_70.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(num_trades)
print(sum(pd.DataFrame(trade_log).iloc[:, 4:5].values))

In [ ]:
quantile_30_predictions = np.array(pd.DataFrame(Q_30.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Q_70.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(num_trades)
print(sum(pd.DataFrame(trade_log).iloc[:, 4:5].values))

In [ ]:
quantile_30_predictions = np.array(pd.DataFrame(Q_10.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Q_90.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(num_trades)
print(sum(pd.DataFrame(trade_log).iloc[:, 4:5].values))

In [ ]:
quantile_30_predictions = np.array(pd.DataFrame(Q_50.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Q_50.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(num_trades)
print(sum(pd.DataFrame(trade_log).iloc[:, 4:5].values))

In [ ]:
quantile_30_predictions = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(num_trades)
print(sum(pd.DataFrame(trade_log).iloc[:, 4:5].values))

In [ ]:
date_format="%m/%d/%Y %H:%M"
date_parse = lambda date: dt.datetime.strptime(date, date_format)
# dat = pd.read_csv("/home/ciaran/knn_Q_1-12.csv")
# dat = pd.read_csv("/home/ciaran/lasso_Q_1-12.csv")
dat = pd.read_csv("/home/ciaran/lgbm_Q_1-12.csv")
# dat = pd.read_csv("/home/ciaran/rf_Q_1-12.csv")
# dat = pd.read_csv("/home/ciaran/SH_Q_1-12.csv")
# dat = pd.read_csv("/home/ciaran/MH_Q_1-12.csv")

dat1=pd.DataFrame(dat)

column_names = ['lag_{}y_Forecast_10'.format(i) for i in range(2, 18)]
Q_10=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_30'.format(i) for i in range(2, 18)]
Q_30=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_50'.format(i) for i in range(2, 18)]
Q_50=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_70'.format(i) for i in range(2, 18)]
Q_70=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y_Forecast_90'.format(i) for i in range(2, 18)]
Q_90=dat1[column_names].dropna().stack().reset_index()
column_names = ['lag_{}y'.format(i) for i in range(2, 18)]
Y_r=dat1[column_names].dropna().stack().reset_index()

In [ ]:
def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 1

    num_trades = 0
    last_sell_timestamp = 0
    units = 0

    for i in range(len(quantile_70_predictions)):
        if i < last_sell_timestamp:
            continue

        for j in range(1, len(quantile_70_predictions)-i):
            if j > 48:
                break

            if quantile_70_predictions[i] < quantile_30_predictions[i+j]:
                purchase_price = real_prices[i][0]
                sell_price = real_prices[i+j][0]
                
                units = min(max_units, abs(funds // purchase_price))
                profit = (units * (sell_price - purchase_price))*0.8
                
                profits += profit
                funds = funds + profit
                
                trade_log.append((i, i+j, purchase_price, sell_price, units, profit))
                
                num_trades += 1
                last_sell_timestamp = i+j
                break

        else:
            units = 0

    return profits, trade_log, num_trades, units, funds


quantile_30_predictions = np.array(pd.DataFrame(Q_10.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Q_90.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(num_trades)
print(sum(pd.DataFrame(trade_log).iloc[:, 4:5].values))
# print(funds)
# sum(pd.DataFrame(trade_log).iloc[:, 5:].values)

In [ ]:
quantile_30_predictions = np.array(pd.DataFrame(Q_10.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Q_90.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(num_trades)
print(sum(pd.DataFrame(trade_log).iloc[:, 4:5].values))

In [ ]:
quantile_30_predictions = np.array(pd.DataFrame(Q_30.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Q_70.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(num_trades)
print(sum(pd.DataFrame(trade_log).iloc[:, 4:5].values))

In [ ]:
quantile_30_predictions = np.array(pd.DataFrame(Q_50.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Q_50.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(num_trades)
print(sum(pd.DataFrame(trade_log).iloc[:, 4:5].values))

In [ ]:
quantile_30_predictions = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(num_trades)
print(sum(pd.DataFrame(trade_log).iloc[:, 4:5].values))

In [ ]:
pd.DataFrame(trade_log)

In [ ]:
pd.DataFrame(trade_log).head(n=39)

In [ ]:
def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 10

    num_trades = 0
    latest_buy_timestamp = 0

    for i in range(0, len(quantile_70_predictions), 24):
        max_profit = 0
        best_buy_timestamp = i
        best_sell_timestamp = i

        for j in range(i, min(i+24, len(quantile_70_predictions))):
            if j <= latest_buy_timestamp:
                continue

            for k in range(j, min(i+24, len(quantile_70_predictions))):
                if quantile_70_predictions[j] < quantile_30_predictions[k]:
                    purchase_price = real_prices[j]
                    sell_price = real_prices[k]
                    units = min(max_units, abs(funds // purchase_price))
                    profit = units * (sell_price - purchase_price) * 0.8

                    if profit > max_profit:
                        max_profit = profit
                        best_buy_timestamp = j
                        best_sell_timestamp = k

        if max_profit > 0:
            purchase_price = real_prices[best_buy_timestamp]
            sell_price = real_prices[best_sell_timestamp]
            units = min(max_units, abs(funds // purchase_price))
            profit = units * (sell_price - purchase_price) * 0.8

            profits += profit
            funds = funds

            trade_log.append((best_buy_timestamp, best_sell_timestamp, purchase_price, sell_price, units, profit))

            num_trades += 1
            latest_buy_timestamp = best_buy_timestamp

    return profits, trade_log, num_trades, units, funds



def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 10

    num_trades = 0
    latest_buy_timestamp = 0

    for i in range(0, len(quantile_70_predictions), 24):
        max_profit = 0
        best_buy_timestamp = i
        best_sell_timestamp = i

        for j in range(i, min(i+24, len(quantile_70_predictions))):
            if j <= latest_buy_timestamp:
                continue

            if quantile_70_predictions[best_buy_timestamp] < quantile_30_predictions[j]:
                purchase_price = real_prices[best_buy_timestamp]
                sell_price = real_prices[j]
                units = min(max_units, abs(funds // purchase_price))
                profit = units * (sell_price - purchase_price) * 0.8

                if profit > max_profit:
                    max_profit = profit
                    best_sell_timestamp = j

        if max_profit > 0:
            purchase_price = real_prices[best_buy_timestamp]
            sell_price = real_prices[best_sell_timestamp]
            units = min(max_units, abs(funds // purchase_price))
            profit = units * (sell_price - purchase_price) * 0.8

            profits += profit
            funds = funds

            trade_log.append((best_buy_timestamp, best_sell_timestamp, purchase_price, sell_price, units, profit))

            num_trades += 1
            latest_buy_timestamp = best_buy_timestamp
            

    return profits, trade_log, num_trades, units, funds



def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 10

    num_trades = 0
    latest_buy_timestamp = 0

    for i in range(0, len(quantile_70_predictions), 24):
        for j in range(i+1, min(i+24, len(quantile_70_predictions))):
            if j <= latest_buy_timestamp:
                continue
            if quantile_70_predictions[i] < quantile_30_predictions[j]:
                purchase_price = real_prices[i]
                sell_price = real_prices[j]
                
                units = min(max_units, abs(funds // purchase_price))
                profit = units * (sell_price - purchase_price) * 0.8
                
                profits += profit
                funds += profit
                
                trade_log.append((i, j, purchase_price, sell_price, units, profit))
                
                num_trades += 1
                latest_buy_timestamp = i
                
                break

    return profits, trade_log, num_trades, units, funds

In [ ]:
def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 10

    num_trades = 0
    last_sell_timestamp = 0

    for i in range(0, len(quantile_70_predictions), 24):
        for j in range(i+1, min(i+24, len(quantile_70_predictions))):
            if quantile_70_predictions[i] < quantile_30_predictions[j]:
                
                purchase_price = real_prices[i]
                sell_price = real_prices[j]
                
                units = min(max_units, abs(funds // purchase_price))
                profit = units * (sell_price - purchase_price) * 0.8
                
                profits += profit
                funds = funds + profit
                
                trade_log.append((i, j, purchase_price, sell_price, units, profit))
                
                num_trades += 1
                last_sell_timestamp = j
                
                break

    return profits, trade_log, num_trades, units, funds




quantile_30_predictions = np.array(pd.DataFrame(Q_50.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Q_50.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(num_trades)
print(sum(pd.DataFrame(trade_log).iloc[:, 4:5].values))
# print(units)
# print(funds)
# sum(pd.DataFrame(trade_log).iloc[:, 5:].values)

In [ ]:
pd.DataFrame(trade_log).head(n=20)

working attempt where we can restrict the appraoch to purchase when the 70th quantile prediction at time point i is less than the 30th quantile prediction at a later time point i+j.

No optimisation used here, this is just representative of a working example. Optimisation approach included further down.

In [ ]:
def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 10

    num_trades = 0
    last_sell_timestamp = 0

    for i in range(len(quantile_70_predictions)):
        if i < last_sell_timestamp:
            continue

        for j in range(1, len(quantile_70_predictions)-i):
            
            if quantile_70_predictions[i] < quantile_30_predictions[i+j]:
                
                purchase_price = real_prices[i]
                sell_price = real_prices[i+j]
                
                units = min(max_units, abs(funds // purchase_price))
                profit = units * (sell_price - purchase_price)*0.8
                
                profits += profit
                funds = funds
                
                trade_log.append((i, i+j, purchase_price, sell_price, units, profit))
                
                num_trades += 1
                last_sell_timestamp = i+j
                
                break

    return profits, trade_log, num_trades, units, funds


quantile_30_predictions = np.array(pd.DataFrame(Q_10.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Q_70.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(num_trades)
# print(units)
# print(funds)
# sum(pd.DataFrame(trade_log).iloc[:, 5:].values)

In [ ]:
pd.DataFrame(trade_log)

In [ ]:
sum(pd.DataFrame(trade_log).iloc[:, 5:].values)

In [ ]:
pd.DataFrame(trade_log)

Of interest, increasing the funds also changes the multiple of profits, likely as funds is not constrained by unit
size or quantity like profits are, ignoring our effect on the price.

what I'll do is collect predictions for the DAM over the same time period with the same method to see if it
makes a sunstantial difference on either trade frequence or profits levels compared to the BM

potentiall because the market is so volatile the models are learning to take a wider position on uncertainty 
forcing them to trade far less. This potentially may not be the case in the DAM allowing us to trade with greater frequency but will require a constraint that all trading periods are within each 24 hour time frame




Pasted Below is a comparison of trading off of all different quantile pairs. No constraint on maximum units, first row represents total profits, second row total trades, third row number of units purchased


In [ ]:
#0.1,0.3
[2800.17]
8
[12.]
#0.1,0.5
[700.68]
5
[3.]
#0.1,0.7
[3645.99]
4
[15.]
#0.1,0.9
[1117.]
2
[9.]

#0.3,0.5
[1588.93]
9
[7.]
#0.3,0.7
[1099.07]
14
[5.]
#0.3,0.9
[6202.04]
6
[25.]

#0.5,0.7
[2516.54]
16
[11.]
#0.5,0.9
[33864.86]
11
[133.]

#0.7,0.9
[4609.31]
17
[20.]

Pasted Below: maximum units 1, first row total profits, second row total trades


In [ ]:
#0.1,0.3
[405.58]
8
#0.1,0.5
[320.14]
5
#0.1,0.7
[304.89]
4
#0.1,0.9
[377.55]
2

#0.3,0.5
[484.33]
9
#0.3,0.7
[356.7]
14
#0.3,0.9
[429.75]
6

#0.5,0.7
[368.96]
16
#0.5,0.9
[854.84]
11

#0.7,0.9
[532.49]
17

Taken from google on the effects of the optimsation approach.

Using optimisation even though there will be no change in the output with the current weights of [1,1],however varying these constraints 'x0' which represents the initial value for the optimization algorithm's decision variables. In this case, the decision variables are the weights [1, 1]. The optimization algorithm will adjust these weights to maximize the profits returned by the calculate_profits() function.

The weights determine the criteria for when to buy and sell. If the weights are set to a certain value, the algorithm will be more or less likely to trade based on those criteria. For example, if w_0 is set to a very high value, the algorithm may be more likely to trade because it is taking into account the predictions of the upper quantile more heavily. This could result in a higher trading frequency.

This may be well suited to a hyper paramater search for optimal weighting for the trading portion.

In [ ]:
import numpy as np
from scipy.optimize import minimize

def calculate_profits(weights, quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 1

    num_trades = 0
    last_sell_timestamp = 0

    for i in range(len(quantile_70_predictions)):
        if i < last_sell_timestamp:
            continue

        for j in range(1, len(quantile_70_predictions)-i):
            if quantile_70_predictions[i] * weights[0] < quantile_30_predictions[i+j] * weights[1]:
                purchase_price = real_prices[i]
                sell_price = real_prices[i+j]
                
                units = min(max_units, funds // purchase_price)
                profit = units * (sell_price - purchase_price)
                
                profits += profit
                funds = (sell_price/purchase_price) * funds
                
                trade_log.append((i, i+j, purchase_price, sell_price, units, profit))
                
                num_trades += 1
                last_sell_timestamp = i+j
                
                break

    return profits, trade_log, num_trades, units, funds

#lower quantile
quantile_30_predictions = np.array(pd.DataFrame(Q_10.iloc[:, 2:]))
#upper quantile
quantile_70_predictions = np.array(pd.DataFrame(Q_90.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))


def objective(weights, quantile_70_predictions, quantile_30_predictions, real_prices):
    profits, trade_log, num_trades, units, funds = calculate_profits(weights, quantile_70_predictions, quantile_30_predictions, real_prices)
    return -profits

x0 = [3, 1]

res = minimize(objective, x0, args=(quantile_70_predictions, quantile_30_predictions, real_prices), method='SLSQP')

profits, trade_log, num_trades, units, funds = calculate_profits(x0, quantile_70_predictions, quantile_30_predictions, real_prices)

# print(res.x)
print(-res.fun)
print(num_trades)
print(units)
# print(profits)
trade_log


With a added threshold paramater added so to only trade when expected profit or loss is within a certain range

In [ ]:
def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices, threshold):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 1

    num_trades = 0
    last_sell_timestamp = 0

    for i in range(len(quantile_70_predictions)):
        if i < last_sell_timestamp:
            continue

        for j in range(1, len(quantile_70_predictions)-i):
            purchase_price = real_prices[i]
            sell_price = real_prices[i+j]
            expected_profit = (quantile_30_predictions[i+j] - quantile_70_predictions[i]) / quantile_70_predictions[i]

            # Only execute trade if expected profit or loss is within the threshold
            if quantile_70_predictions[i] < quantile_30_predictions[i+j] and abs(expected_profit) <= threshold:
                units = min(max_units, funds // purchase_price)
                profit = units * (sell_price - purchase_price)
                
                profits += profit
                funds = (sell_price/purchase_price) * funds
                
                trade_log.append((i, i+j, purchase_price, sell_price, units, profit))
                
                num_trades += 1
                last_sell_timestamp = i+j
                
                break

    return profits, trade_log, num_trades, units, funds


quantile_30_predictions = np.array(pd.DataFrame(Q_50.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Q_50.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices, 0.1)
print(total_profits)
print(num_trades)
print(units)
print(funds)

trade_log

In [ ]:
#pymo

Plot below should highlight its a very rare occurence for the 0.9 quantile at time point i to be below the 0.1 quantile at time point i+j.

Point stands for quantile 0.7 and 0.3

In [ ]:
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import os
from matplotlib.ticker import ScalarFormatter,AutoMinorLocator
import matplotlib as mpl

plt.style.use('plot_style.txt')
#=========================================================== # Directory and filename; style file open #=========================================================== # Change to the directory which contains the current script 
dirFile = os.path.dirname(os.path.join('YourDirHere',
                          'NicePlotProductivity.py'))
# Load style file plt.style.use(os.path.join(dirFile, 'PaperDoubleFig.mplstyle'))
# Make some style choices for plotting 
colourWheel =['#329932',
            '#ff6961',
            'b',
            '#6a3d9a',
            '#fb9a99',
            '#e31a1c',
            '#fdbf6f',
            '#ff7f00',
            '#cab2d6',
            '#6a3d9a',
            '#ffff99',
            '#b15928',
            '#67001f',
            '#b2182b',
            '#d6604d',
            '#f4a582',
            '#fddbc7',
            '#f7f7f7',
            '#d1e5f0',
            '#92c5de',
            '#4393c3',
            '#2166ac',
            '#053061']
dashesStyles = [[3,1],
            [1000,1],
            [2,1,10,1],
            [4, 1, 1, 1, 1, 1]]





p1=Q_10.iloc[:, 2:]
p1=p1.reset_index(drop=True)
p3=Q_30.iloc[:, 2:]
p3=p3.reset_index(drop=True)
p5=Q_50.iloc[:, 2:]
p5=p5.reset_index(drop=True)
p7=Q_70.iloc[:, 2:]
p7=p7.reset_index(drop=True)
p9=Q_90.iloc[:, 2:]
p9=p9.reset_index(drop=True)
yy=Y_r.iloc[:, 2:]
yy=yy.reset_index(drop=True)

p1=p1.values.ravel()
p3=p3.values.ravel()
p5=p5.values.ravel()
p7=p7.values.ravel()
p9=p9.values.ravel()
                       
figure(figsize = (30, 20), dpi = 90)
plt.plot(yy, color='black', label='BM prices', alpha=0.3)
plt.plot(p1, color='blue', label='predictions 0.1 quantile', alpha=0.3)
# plt.plot(p3, color='yellow', label='SH DNN predictions 0.3 quantile')
# plt.plot(p5, color='red', label='SH DNN predictions 0.5 quantile')
# plt.plot(p7, color='yellow', label='SH DNN predictions 0.7 quantile')
plt.plot(p9, color='blue', label='predictions 0.9 quantile', alpha=0.3)
plt.fill_between(np.arange(0,len(yy),1), p1, p9, alpha=0.2)


plt.xlabel('0.1, 0.9 quantiles Forecast period from 01/06/20-01/06/21',fontsize=18)
plt.ylabel('€/MWh', fontsize=18)
plt.title("Forecast of BM prices", fontsize=18)
plt.xlim([-10, 3000])
plt.ylim([-50,200])
plt.legend()
# plt.savefig('SH_DNN.png', transparent=False, format="jpg")


In [ ]:
figure(figsize = (30, 20), dpi = 90)
plt.plot(yy, color='black', label='BM prices', alpha=0.3)
plt.plot(p3, color='blue', label='predictions 0.3 quantile', alpha=0.3)
# plt.plot(p3, color='yellow', label='SH DNN predictions 0.3 quantile')
# plt.plot(p5, color='red', label='SH DNN predictions 0.5 quantile')
# plt.plot(p7, color='yellow', label='SH DNN predictions 0.7 quantile')
plt.plot(p7, color='blue', label='predictions 0.7 quantile', alpha=0.3)
plt.fill_between(np.arange(0,len(yy),1), p3, p7, alpha=0.2)


plt.xlabel('0.3, 0.7 quantiles Forecast period from 01/06/20-01/06/21',fontsize=18)
plt.ylabel('€/MWh', fontsize=18)
plt.title("Forecast of BM prices", fontsize=18)
plt.xlim([-10, 3000])
plt.ylim([-50,200])
plt.legend()
# plt.savefig('SH_DNN.png', transparent=False, format="jpg")


Everthing below contains previous attempt at the current working code or is junk working copies adapted from articles on trading.

In [ ]:
# def objective_function(w, q30, q70, prices):
#   profits = []

#   last_trade_timestamp = -1

#   trade_threshold = 0.5

#   for i, price in enumerate(prices[:-1]):
#     for j in range(1, len(q30) - i):
#       if q70[i] < q30[i + j] and i + j > last_trade_timestamp:  
#         if all(q70[i] < q70[i + j + k] for k in range(1, len(q30) - (i + j))):
#           purchase_price = prices[i]
#           sell_price = prices[i + j]

#           if sell_price > (1 + trade_threshold) * purchase_price:
#             continue  

#           profit = sell_price - purchase_price

#           profits.append(profit)
#           global funds, trades  
#           funds = funds + (sell_price - purchase_price)
#           trades += 1 
        
#           last_trade_timestamp = i + j

#   total_profit = sum(profits)


#   return -total_profit
def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 1000
    profits = 0
    trade_log = []
    max_units = 1
    trade_threshold = 0.9


    num_trades = 0
    last_sell_timestamp = 0

    for i in range(len(quantile_70_predictions)):
        if i < last_sell_timestamp:
            continue

        for j in range(1, len(quantile_70_predictions)-i):
            
            if quantile_70_predictions[i] < quantile_30_predictions[i+j]:
                
                purchase_price = real_prices[i]
                sell_price = real_prices[i+j]
                if sell_price > (1 + trade_threshold) * purchase_price:
                    continue 
                
                units = min(max_units, funds // purchase_price)
                profit = units * (sell_price - purchase_price)
                
                profits += profit
                funds = (sell_price/purchase_price) * funds
                
                trade_log.append((i, i+j, purchase_price, sell_price, units, profit))
                
                num_trades += 1
                last_sell_timestamp = i+j
                
                break

    return profits, trade_log, num_trades, units, funds

quantile_30_predictions = np.array(pd.DataFrame(Q_50.iloc[:, 2:]))
quantile_70_predictions = np.array(pd.DataFrame(Q_50.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades, units, funds = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(num_trades)
print(units)
print(funds)

trade_log

In [ ]:
# dat1 = df.loc[(df['lag_2y'] < -5) | (df['lag_2y'] > 5)]
# dat1 = df.loc[(df.iloc[:,:16] > 5)]
# dat1 = df.loc[(df.iloc[:,:16] > 1).all(axis=1)]
dat.shape

dat1=df

In [ ]:
def optimize_trades(quantile_70_predictions, quantile_30_predictions, real_prices, max_units):
    prob = pulp.LpProblem('Trade Optimization', pulp.LpMaximize)
    
    x = pulp.LpVariable.dicts('trade', range(len(quantile_70_predictions)), lowBound=0, upBound=1, cat='Binary')

    prob += pulp.lpSum([x[i] * (real_prices[i+j] - real_prices[i]) for i in range(len(quantile_70_predictions)) for j in range(1, len(quantile_70_predictions)-i) if quantile_70_predictions[i] < quantile_30_predictions[i+j]])
    prob += pulp.lpSum([x[i] for i in range(len(quantile_70_predictions))]) <= max_units

    status = prob.solve()
    print(pulp.LpStatus[status])
    for v in prob.variables():
        print(v.name, '=', v.varValue)
    print('Total profits =', pulp.value(prob.objective))

optimize_trades(quantile_70_predictions, quantile_30_predictions, real_prices, 10)



In [ ]:
# Declare the funds and trades variables as global variables
funds = 1000
trades = 0
trade_log = []  # Declare a list variable to store the trades

def objective_function(w, q30, q70, prices):
  profits = []

  last_trade_timestamp = -1

  max_units = 1

  global funds, trades

  for i, price in enumerate(prices[:-1]):
    for j in range(1, len(q30) - i):
      if q70[i] < q30[i + j] and i + j > last_trade_timestamp:  # Check that the sell time stamp is after the last trade timestamp
        if all(q70[i] < q70[i + j + k] for k in range(1, len(q30) - (i + j))):
          purchase_price = prices[i]
          sell_price = prices[i + j]
          
#           if sell_price <= 0:
#             continue

          units = min(max_units, funds // purchase_price)
          profit = units * (sell_price - purchase_price)
            
#           if profit <= 0:
#             continue

          profits.append(profit)
          funds += profit
          trades += 1
          trade_log.append((purchase_price, sell_price, units, profit))


          last_trade_timestamp = i + j

  total_profit = sum(profits)


  return -total_profit




df_q30 = pd.DataFrame(Q_10.iloc[:, 2:])
df_q70 = pd.DataFrame(Q_90.iloc[:, 2:])
df_prices = pd.DataFrame(Y_r.iloc[:, 2:])

# Test the objective function with the DataFrame objects
w0 = [1, 1]  # Initial values for the model parameters
res = minimize(objective_function, w0, args=(np.array(df_q30), np.array(df_q70), np.array(df_prices)))

# Print the optimized model parameters, the final value of the funds, and the number of trades
print(f'Optimized model parameters: {res.x}')
print(f'Final value of the funds: {funds}')
print(f'Number of trades: {trades}')
print(f'Number of trades: {trade_log}')

In [ ]:
trade_log

In [ ]:
# Declare the funds and trades variables as global variables
funds = 100
trades = 0
trade_log = []  # Declare a list variable to store the trades

def objective_function(q30, q70, prices):
  profits = []

  last_trade_timestamp = -1

  max_units = 1

  global funds, trades

  for i, price in enumerate(prices[:-1]):
    for j in range(1, len(q30) - i):
      if q70[i] < q30[i + j] and i + j > last_trade_timestamp:  
        if all(q70[i] < q70[i + j + k] for k in range(1, len(q30) - (i + j))):
          purchase_price = prices[i]
          sell_price = prices[i + j]
          
          units = min(max_units, funds // purchase_price)
          profit = units * (sell_price - purchase_price)
            
          profits.append(profit)
          funds += profit
          trades += 1

          trade_log.append((purchase_price, sell_price, units, profit))
          
          last_trade_timestamp = i + j

  total_profit = sum(profits)


  return -total_profit


def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices, max_units):
    funds = 100
    profits = 0
    trade_log = []

    num_trades = 0
    last_sell_timestamp = 0

    for i in range(len(quantile_70_predictions)):
        if i < last_sell_timestamp:
            continue

        for j in range(1, len(quantile_70_predictions)-i):
            if quantile_70_predictions[i] < quantile_30_predictions[i+j]:
                purchase_price = real_prices[i]
                sell_price = real_prices[i+j]
                units = min(max_units, funds // purchase_price)
                profit = units * (sell_price - purchase_price)
                profits += profit
                funds = (sell_price/purchase_price) * funds
                trade_log.append((i, j, purchase_price, sell_price, units, profit))
                num_trades += 1
                last_sell_timestamp = i+j
                break

    return profits




# df_q30 = pd.DataFrame(Q_10.iloc[:, 2:])
# df_q70 = pd.DataFrame(Q_90.iloc[:, 2:])
# df_prices = pd.DataFrame(Y_r.iloc[:, 2:])

df_q30 =    [100, 110, 120, 130, 140, 150, 160]
df_q70 =    [90,  100, 150, 120, 190, 140, 190]
df_prices = [100,  60, 120, 30, 140, 150, 60]

objective_function(np.array(df_q30), np.array(df_q70), np.array(df_prices))

print(f'Final value of the funds: {funds}')
print(f'Number of trades: {trades}')

# print(f'Trade log: {trade_log}')


In [ ]:
trade_log
#purchase_price, sell_price, units, profit

In [ ]:
trade_log

In [ ]:
def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 100
    profits = 0
    trade_log = []

    num_trades = 0
    last_sell_timestamp = 0

    for i in range(len(quantile_70_predictions)):
        if i < last_sell_timestamp:
            continue

        for j in range(1, len(quantile_70_predictions)-i):
            if quantile_70_predictions[i] < quantile_30_predictions[i+j]:
                purchase_price = real_prices[i]
                sell_price = real_prices[i+j]
                profits += sell_price - purchase_price
                funds = (sell_price/purchase_price) * funds
                trade_log.append((i, j, purchase_price, sell_price))
                num_trades += 1
                last_sell_timestamp = i+j
                break

    return profits, trade_log, num_trades


quantile_70_predictions = np.array(pd.DataFrame(Q_90.iloc[:, 2:]))
quantile_30_predictions = np.array(pd.DataFrame(Q_10.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
print(trade_log)
print(num_trades)


In [ ]:
trade_log

In [ ]:
def calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices):
    funds = 100
    profits = 0
    trade_log = []
    max_units = 1


    num_trades = 0
    last_sell_timestamp = 0

    for i in range(len(quantile_70_predictions)):
        if i < last_sell_timestamp:
            continue

        for j in range(1, len(quantile_70_predictions)-i):
            if quantile_70_predictions[i] < quantile_30_predictions[i+j]:
                purchase_price = real_prices[i]
                sell_price = real_prices[i+j]
                units = min(max_units, funds // purchase_price)
                profit = units * (sell_price - purchase_price)
                profits += profit
                funds = (sell_price/purchase_price) * funds
                trade_log.append((i, j, purchase_price, sell_price, units, profit))
                num_trades += 1
                last_sell_timestamp = i+j
                break

    return profits, trade_log, num_trades

quantile_70_predictions = np.array(pd.DataFrame(Q_90.iloc[:, 2:]))
quantile_30_predictions = np.array(pd.DataFrame(Q_10.iloc[:, 2:]))
real_prices = np.array(pd.DataFrame(Y_r.iloc[:, 2:]))

total_profits, trade_log, num_trades = calculate_profits(quantile_70_predictions, quantile_30_predictions, real_prices)
print(total_profits)
# print(trade_log)
print(num_trades)
trade_log

In [ ]:
trade_log

In [ ]:
funds = 1000
trades = 0

def objective_function(w, q30, q70, prices):
  profits = []

  last_trade_timestamp = -1

  trade_threshold = 0.5

  for i, price in enumerate(prices[:-1]):
    for j in range(1, len(q30) - i):
      if q70[i] < q30[i + j] and i + j > last_trade_timestamp:  
        if all(q70[i] < q70[i + j + k] for k in range(1, len(q30) - (i + j))):
          purchase_price = prices[i]
          sell_price = prices[i + j]

          if sell_price > (1 + trade_threshold) * purchase_price:
            continue  

          profit = sell_price - purchase_price

          profits.append(profit)
          global funds, trades  
          funds = funds + (sell_price - purchase_price)
          trades += 1 
        
          last_trade_timestamp = i + j

  total_profit = sum(profits)


  return -total_profit

df_q30 = pd.DataFrame(Q_30.iloc[:, 2:])
df_q70 = pd.DataFrame(Q_70.iloc[:, 2:])
df_prices = pd.DataFrame(Y_r.iloc[:, 2:])

w0 = [1, 1]  
res = minimize(objective_function, w0, args=(np.array(df_q30), np.array(df_q70), np.array(df_prices)))

print(f'Optimized model parameters: {res.x}')
print(f'Final value of the funds: {funds}')
print(f'Number of trades: {trades}')

In [ ]:

funds = 1000
trades = 0

def objective_function(w, q30, q70, prices):
  profits = []

  last_trade_timestamp = -1

  for i, price in enumerate(prices[:-1]):
    for j in range(1, len(q30) - i):
      if q70[i] < q30[i + j] and i + j > last_trade_timestamp:  
        if all(q70[i] < q70[i + j + k] for k in range(1, len(q30) - (i + j))):
          purchase_price = prices[i]
          sell_price = prices[i + j]

          profit = sell_price - purchase_price

          profits.append(profit)
          global funds, trades  
          funds = (sell_price / purchase_price) * funds
          trades += 1  
        
          last_trade_timestamp = i + j

  total_profit = sum(profits)


  return -total_profit



df_q30 = pd.DataFrame(Q_10.iloc[:, 2:])
df_q70 = pd.DataFrame(Q_90.iloc[:, 2:])
df_prices = pd.DataFrame(Y_r.iloc[:, 2:])

w0 = [1, 1]  
res = minimize(objective_function, w0, args=(np.array(df_q30), np.array(df_q70), np.array(df_prices)))

print(f'Optimized model parameters: {res.x}')
print(f'Final value of the funds: {funds}')
print(f'Number of trades: {trades}')

In [ ]:
p1=Q_10.iloc[:, 2:].values.ravel()
yy= Y_r.iloc[:, 2:].values.ravel()
p9=Q_90.iloc[:, 2:].values.ravel()
                       
figure(figsize = (30, 20), dpi = 90)
plt.plot(yy, color='black', label='BM prices', alpha=0.6)
plt.plot(p1, color='blue', label='SH DNN predictions 0.1 quantile', alpha=0.2)
# plt.plot(p3, color='yellow', label='SH DNN predictions 0.3 quantile')
# plt.plot(p5, color='red', label='SH DNN predictions 0.5 quantile')
# plt.plot(p7, color='yellow', label='SH DNN predictions 0.7 quantile')
plt.plot(p9, color='blue', label='SH DNN predictions 0.9 quantile', alpha=0.2)
plt.fill_between(np.arange(0,len(yy),1), p1, p9, alpha=0.4)
